In [1]:
from core.DataLoader import *
from core.models.MLP import * 
from core.models.GAM import *
from core.models.SVM import *
# from core.models.EBM import *

from core.models.RandomForest import *
from core.Visualizer import *
from core.models.Classifier import *
from core.models.XGB import *

In [2]:
model_accs = {
    'MLP': [],
    'RF(5)': [],
    'RF(10)': [],
    'RF(U)': [],
    'DT(5)': [],
    'DT(10)': [],
    'DT(U)': [],
    'LR': [],
    'SVM': [],
    'XGB': []
}
# present_idxs = [0, 1, 2, 3, 4, 5, 8 ,12, 21, 22, 23, 24]

In [3]:
weighted_clf_model_list ={
    "MLP": MLPClassifier,
    "DT": m_DecisionTreeClassifier,
    "RF": m_RandomForestClassifier,
    "LR": m_LogisticRegression,
    "SVM": SVMClassifier,
    "XGB": XGBOOSTClassifier
}
weighted_clf_model_name_list = ['MLP', 'RF(5)', 'RF(U)', 'DT(5)', 'DT(U)', 'LR', 'SVM', 'XGB']

In [4]:
engine = 'mssql'
dl = DataLoader(engine)
one_file_dss, one_file_names = dl.get_one_file_ds(return_type='ds and names', datasets=['ssb', 'tpch', 'tpch_10', 'tpch_100', 'imdb'], suffix='(random_left)')

In [5]:
key_features = [['sel_of_pred_on_indexed_attr', 'left_cardinality'], 
                ['sel_of_pred_on_indexed_attr_and_join_pred', 'left_cardinality'],
                ['sel_of_pred_on_indexed_attr_and_join_pred', 'sel_of_pred_on_indexed_attr_and_non_indexed_attr'],
               ['sel_of_pred_on_indexed_attr_and_join_pred']]

# if engine == 'mssql':
#     key_features = ['sel_of_pred_on_indexed_attr_and_join_pred', 'left_cardinality']
# else:
#     key_features = ['sel_of_pred_on_indexed_attr', 'left_cardinality']
    
classification_target = dl.classification_target

if engine == 'mssql':
    key_feature_set = 0
else:
    key_feature_set = 0
    
features = key_features[key_feature_set]
print("Using features: ", features)

Using features:  ['sel_of_pred_on_indexed_attr', 'left_cardinality']


In [6]:
# model_perf = {
    
# }

present_idxs = range(len(one_file_dss))

for i in present_idxs:  # range(len(one_file_dss)):
#     i = 0
    print(f"Processing: {i}")
    # ds = one_file_dss[i][one_file_dss[i]['predicate_op_num_on_non_indexed_attr'] == 0]
    ds = one_file_dss[i]
    ds_name = one_file_names[i]
    
    # =========================
    X = ds[features]
    y = ds['optimal_decision']
    X_costs = ds[dl.regression_targets]

    from sklearn.model_selection import train_test_split
    from sklearn import preprocessing

    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)
    X_train_costs, _, _, _, = train_test_split(X_costs, y, train_size=0.8, random_state=1)
    X_train, X_test, y_train, y_test = \
        X_train.to_numpy(),X_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()

    X_train_weights = calculate_importance_from_costs(X_train_costs.to_numpy())

    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)

    # scaler = preprocessing.StandardScaler().fit(X_test)
    X_test = scaler.transform(X_test)

    X_train_weights = preprocessing.MinMaxScaler().fit_transform(X_train_weights.reshape(-1,1)).flatten()
    # =========================
    
    accs = []
    for idx, model_name in enumerate(weighted_clf_model_name_list):
        model = weighted_clf_model_list[model_name.split('(')[0]]        
        if 'DT' in model_name or 'RF' in model_name:
            max_depth = model_name.split('(')[-1].split(')')[0]
            if max_depth.lower() == 'u':
                max_depth = None
            else:
                max_depth = int(max_depth)
            clf = model().fit(X_train, y_train, sample_weight=X_train_weights, max_depth=max_depth)
        elif model_name == 'MLP':
            clf = model().fit(X_train, y_train, sample_weight=X_train_weights, max_iter=500, weight_decay=0.000001)
        else:
            clf = model().fit(X_train, y_train, sample_weight=X_train_weights)
            
        print(f"Accuray of {model}: {clf.score(X_test, y_test)}")
#         if len(features) == 2:
#             plot_2d_decision_boundaries(clf, scaler.inverse_transform(X_train), X_train_costs, y_train, title=f'{model} on {one_file_names[i]}')
        
        model_accs[model_name].append(clf.score(X_test, y_test))
#         accs.append(clf.score(X_test, y_test))
        
    
#     model_perf[ds_name] = accs
#     break
    

  0%|          | 0/500 [00:00<?, ?it/s]

Processing: 0


100%|██████████| 500/500 [01:07<00:00,  7.46it/s]


Accuray of <class 'core.models.MLP.MLPClassifier'>: 0.8475
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 0.9425
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 0.9825
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 0.9475
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 0.9775
Accuray of <class 'core.models.GAM.m_LogisticRegression'>: 0.8525


  0%|          | 0/500 [00:00<?, ?it/s]

...

Accuray of <class 'core.models.MLP.MLPClassifier'>: 0.9575
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 0.96
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 0.9575
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 0.96
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 0.955
Accuray of <class 'core.models.GAM.m_LogisticRegression'>: 0.9225
Accuray of <class 'core.models.SVM.SVMClassifier'>: 0.9225


  1%|          | 3/500 [00:00<00:18, 26.20it/s]

Accuray of <class 'core.models.XGB.XGBOOSTClassifier'>: 0.9225
Processing: 62


100%|██████████| 500/500 [00:20<00:00, 24.11it/s]


Accuray of <class 'core.models.MLP.MLPClassifier'>: 0.9775
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 0.985
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 1.0
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 0.975
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 0.99
Accuray of <class 'core.models.GAM.m_LogisticRegression'>: 0.9775
Accuray of <class 'core.models.SVM.SVMClassifier'>: 0.9775
Accuray of <class 'core.models.XGB.XGBOOSTClassifier'>: 0.9775


# Plot the graph

In [7]:
# import numpy as np
# import matplotlib.pyplot as plt

# import matplotlib
# matplotlib.rcParams['pdf.fonttype'] = 42
# matplotlib.rcParams['ps.fonttype'] = 42

# # SetPlotRC()
# # set width of bar
# barWidth = 0.4
 
# # SELECT THE FIRST 4 ROWS of each dataset
# # set height of bar

# MLP = model_accs['MLP'] # [97,96.5,97.5,96.5,96.5,91.5,96.5,96,100,98.5,100,100]
# RF_5 = model_accs['RF(5)'] #[95,95.5,95,96.5,98,90.5,95.5,97,100,98.5,100,100]
# # RF_10 =model_accs['RF(10)'] # [95.5, 98,97.5,98,96.5,94.5,98.5,95.5,100,99,100,100]
# RF_U = model_accs['RF(U)'] #[97.5,97,98.5,98,99,94,97.5,97.5,100,99.5,100,100]
# DT_5 = model_accs['DT(5)'] #[97,97.5,96,97.5,97,90,93,96.5,100,99.5,100,100]
# # DT_10 = model_accs['DT(10)'] #[98,98.5,96.5,96.5,97,91.5,95.5,95.5,100,98.5,100,100]
# DT_U = model_accs['DT(U)'] #[98,98.5,96.5,96.5,97,92,95,95.5,100,99,100,100]
# LR = model_accs['LR'] #[92,83.5,92,94,89.5,91,92,90.5,100,96,100,100]
# SVM = model_accs['SVM'] #[85,83,88.5,88,82.5,89,85.5,86.5,100,95.5,100,100]
# XGB = model_accs['XGB'] #[85,83,88.5,88,82.5,89,85.5,86.5,100,95.5,100,100]

# # MLP = model_accs['MLP'] # [97,96.5,97.5,96.5,96.5,91.5,96.5,96,100,98.5,100,100]
# # RF_5 = model_accs['RF(5)'] #[95,95.5,95,96.5,98,90.5,95.5,97,100,98.5,100,100]
# # RF_10 =model_accs['RF(10)'] # [95.5, 98,97.5,98,96.5,94.5,98.5,95.5,100,99,100,100]
# # RF_U = model_accs['RF(U)'] #[97.5,97,98.5,98,99,94,97.5,97.5,100,99.5,100,100]
# # DT_5 = model_accs['DT(5)'] #[97,97.5,96,97.5,97,90,93,96.5,100,99.5,100,100]
# # DT_10 = model_accs['DT(10)'] #[98,98.5,96.5,96.5,97,91.5,95.5,95.5,100,98.5,100,100]
# # DT_U = model_accs['DT(U)'] #[98,98.5,96.5,96.5,97,92,95,95.5,100,99,100,100]
# # LR = model_accs['LR'] #[92,83.5,92,94,89.5,91,92,90.5,100,96,100,100]
# # SVM = model_accs['SVM'] #[85,83,88.5,88,82.5,89,85.5,86.5,100,95.5,100,100]
 
# # Set position of bar on X axis
# r1 = [i*6 for i in range(12)]
# r2 = [x + barWidth for x in r1]
# r3 = [x + barWidth for x in r2]
# r4 = [x + barWidth for x in r3]
# r5 = [x + barWidth for x in r4]
# r6 = [x + barWidth for x in r5]
# r7 = [x + barWidth for x in r6]
# r8 = [x + barWidth for x in r7]
# r9 = [x + barWidth for x in r8]
# r10 = [x + barWidth for x in r9]

# plt.figure(figsize=(40, 5), dpi=300)

# axes = plt.gca()
# # axes.set_xlim([xmin,xmax])
# axes.set_ylim([0, 1])
# plt.grid(axis='y')
# # Make the plot
# plt.bar(r1, MLP, color='tab:blue', width=barWidth, edgecolor='white', label='MLP')
# plt.bar(r2, RF_5, color='tab:orange', width=barWidth, edgecolor='white', label='RF(5)')
# # plt.bar(r3, RF_10, color='tab:green', width=barWidth, edgecolor='white', label='RF(10)')
# plt.bar(r4, RF_U, color='tab:red', width=barWidth, edgecolor='white', label='RF(U)')
# plt.bar(r5, DT_5, color='tab:purple', width=barWidth, edgecolor='white', label='DT(5)')
# # plt.bar(r6, DT_10, color='tab:brown', width=barWidth, edgecolor='white', label='DT(10)')
# plt.bar(r7, DT_U, color='tab:pink', width=barWidth, edgecolor='white', label='DT(U)')
# plt.bar(r8, LR, color='tab:gray', width=barWidth, edgecolor='white', label='LR')
# plt.bar(r9, SVM, color='tab:olive', width=barWidth, edgecolor='white', label='SVM')
# plt.bar(r10, XGB, color='tab:cyan', width=barWidth, edgecolor='white', label='XGB')
 
# # Add xticks on the middle of the group bars
# # plt.xlabel('', fontweight='bold')
# plt.xticks([r + 5*barWidth for r in r1], ['SSB: part-lineorder', 'SSB: customer-lineorder', 'SSB: ddate-lineorder', 
#                                                        'SSB: supplier-lineorder', 'TPC-H: customer-orders', 'TPC-H: orders-customer',
#                                                        'TPC-H: orders-lineitem', 'TPC-H: part-lineitem',
#                                                       'IMDB: title-cast_info', 'IMDB: title-movie_companies', 
#                                                        'IMDB: title-movie_info_idx' , 'IMDB: title-movie_info'])
# axes.tick_params(axis='x', which='major', labelsize=12)
# axes.tick_params(axis='y', which='major', labelsize=20)

# # Create legend & Show graphic
# plt.legend(prop={'size': 10})
# plt.savefig(f'./figures/exp2-ML-key-feat-acc-{engine}-keyset{key_feature_set}.pdf', bbox_inches = 'tight',pad_inches = 0)
# plt.show()

In [8]:
MLP = model_accs['MLP'] # [97,96.5,97.5,96.5,96.5,91.5,96.5,96,100,98.5,100,100]
RF_5 = model_accs['RF(5)'] #[95,95.5,95,96.5,98,90.5,95.5,97,100,98.5,100,100]
# RF_10 =model_accs['RF(10)'] # [95.5, 98,97.5,98,96.5,94.5,98.5,95.5,100,99,100,100]
RF_U = model_accs['RF(U)'] #[97.5,97,98.5,98,99,94,97.5,97.5,100,99.5,100,100]
DT_5 = model_accs['DT(5)'] #[97,97.5,96,97.5,97,90,93,96.5,100,99.5,100,100]
# DT_10 = model_accs['DT(10)'] #[98,98.5,96.5,96.5,97,91.5,95.5,95.5,100,98.5,100,100]
DT_U = model_accs['DT(U)'] #[98,98.5,96.5,96.5,97,92,95,95.5,100,99,100,100]
LR = model_accs['LR'] #[92,83.5,92,94,89.5,91,92,90.5,100,96,100,100]
SVM = model_accs['SVM'] #[85,83,88.5,88,82.5,89,85.5,86.5,100,95.5,100,100]
XGB = model_accs['XGB'] #[85,83,88.5,88,82.5,89,85.5,86.5,100,95.5,100,100]

d = [DT_5, DT_U,  RF_5, RF_U, XGB, MLP, LR, SVM ]
result_txt = ''

for i in range(len(present_idxs)):
    print(f"{one_file_names[i]}", end='\n')
    result_txt += f"{one_file_names[i]} \n"
    for l in d:
        if l[i] >= 0.8951:
            s = '& \\textbf{ %.2f }' % l[i]
        else:
            s = '& %.2f' % l[i]
        result_txt +=  s.replace('0.', '.')
        print(s.replace('0.', '.'), end='')
    print("\n")
    result_txt += '\n'
with open(f'./latex_result/exp2_{engine}_(random_left)_key_feature_acc.txt', 'w') as f:
    f.write(result_txt)

ssb part/mssql_lineorder_part_optimal.csv
& \textbf{ .95 }& \textbf{ .98 }& \textbf{ .94 }& \textbf{ .98 }& \textbf{ .91 }& .85& .85& .85

ssb customer/mssql_lineorder_customer_optimal.csv
& \textbf{ .96 }& \textbf{ .98 }& \textbf{ .96 }& \textbf{ .99 }& \textbf{ .97 }& \textbf{ .98 }& .67& .55

ssb ddate/mssql_lineorder_ddate_optimal.csv
& .88& \textbf{ .96 }& .80& \textbf{ .95 }& .86& .88& .69& .65

ssb supplier/mssql_lineorder_supplier_optimal.csv
& .81& \textbf{ .93 }& .82& \textbf{ .94 }& .81& .84& .75& .75

ssb lineorder/mssql_part_lineorder_optimal.csv
& \textbf{ .99 }& \textbf{ .99 }& \textbf{ .99 }& \textbf{ .99 }& \textbf{ 1.00 }& \textbf{ 1.00 }& \textbf{ 1.00 }& \textbf{ 1.00 }

ssb lineorder/mssql_customer_lineorder_optimal.csv
& \textbf{ 1.00 }& \textbf{ 1.00 }& \textbf{ 1.00 }& \textbf{ 1.00 }& \textbf{ 1.00 }& \textbf{ 1.00 }& \textbf{ 1.00 }& \textbf{ 1.00 }

ssb lineorder/mssql_ddate_lineorder_optimal.csv
& \textbf{ 1.00 }& \textbf{ 1.00 }& \textbf{ 1.00 }& \textbf{ 1

# Latex difference table

In [9]:
# postgres_feature_set_0_acc = [[0.92, 0.985, 0.94, 0.955, 0.905, 0.9325, 0.895, 0.9575, 0.9225, 0.925, 0.9375, 0.9175],
# [0.9275, 0.97, 0.9675, 0.9675, 0.895, 0.94, 0.91, 0.97, 0.9325, 0.925, 0.9275, 0.9425],
# [0.9825, 0.9925, 0.995, 0.99, 0.9775, 0.975, 0.9725, 0.995, 0.9725, 0.975, 0.97, 0.9775],
# [0.93, 0.97, 0.97, 0.955, 0.885, 0.9425, 0.92, 0.9575, 0.9175, 0.9175, 0.925, 0.93],
# [0.9825, 0.99, 0.9975, 0.9925, 0.965, 0.985, 0.98, 1.0, 0.975, 0.975, 0.98, 0.98],
# [0.85, 0.84, 0.9425, 0.93, 0.7475, 0.925, 0.82, 0.945, 0.905, 0.905, 0.915, 0.9075],
# [0.88, 0.87, 0.9125, 0.8725, 0.78, 0.92, 0.875, 0.935, 0.8925, 0.8925, 0.8975, 0.89],
# [0.91, 0.88, 0.94, 0.9375, 0.79, 0.9475, 0.8075, 0.945, 0.9075, 0.905, 0.9075, 0.9175]]

In [10]:
# postgres_feature_set_1_acc = [[0.9175, 0.9525, 0.9525, 0.8175, 0.9075, 0.9375, 0.905, 0.945, 0.92, 0.9375, 0.935, 0.92],
# [0.915, 0.9675, 0.9375, 0.9425, 0.8925, 0.9375, 0.9025, 0.965, 0.9375, 0.9375, 0.93, 0.945],
# [0.97, 0.9925, 0.9775, 0.99, 0.9775, 0.98, 0.9775, 0.9825, 0.9775, 0.97, 0.97, 0.98],
# [0.93, 0.97, 0.9475, 0.8975, 0.885, 0.94, 0.92, 0.965, 0.9175, 0.9175, 0.925, 0.93],
# [0.9825, 0.99, 0.99, 0.995, 0.965, 0.985, 0.98, 0.995, 0.975, 0.975, 0.98, 0.98],
# [0.8175, 0.8375, 0.875, 0.85, 0.7725, 0.925, 0.8225, 0.94, 0.905, 0.905, 0.915, 0.9075],
# [0.87, 0.8725, 0.87, 0.8275, 0.7975, 0.92, 0.8775, 0.945, 0.8925, 0.8925, 0.8975, 0.89],
# [0.82, 0.8075, 0.8725, 0.9125, 0.78, 0.9425, 0.815, 0.9325, 0.9075, 0.905, 0.9075, 0.9175]]

In [11]:
# mssql_feature_set_0_acc = [[0.96, 0.975, 0.855, 0.5975, 0.91, 0.975, 0.7875, 0.94, 0.935, 0.9225, 0.9475, 0.9775],
# [0.955, 0.975, 0.87, 0.6, 0.9125, 0.97, 0.825, 0.9525, 0.915, 0.8975, 0.9625, 0.9625],
# [0.9775, 0.98, 0.88, 0.755, 0.98, 0.9625, 0.955, 0.9575, 0.9925, 0.9875, 0.9875, 0.995],
# [0.965, 0.9725, 0.86, 0.5775, 0.9, 0.965, 0.78, 0.955, 0.925, 0.9375, 0.96, 0.9725],
# [0.97, 0.9825, 0.8775, 0.76, 0.98, 0.9575, 0.9425, 0.9575, 0.9925, 0.9875, 0.985, 0.99],
# [0.865, 0.9125, 0.74, 0.5075, 0.1675, 0.975, 0.805, 0.885, 0.785, 0.7875, 0.8525, 0.875],
# [0.7875, 0.8675, 0.7825, 0.5925, 0.4775, 0.975, 0.845, 0.885, 0.73, 0.81, 0.8925, 0.915],
# [0.8925, 0.91, 0.8325, 0.595, 0.015, 0.975, 0.8075, 0.94, 0.86, 0.87, 0.9525, 0.945]]

In [12]:
# mssql_feature_set_1_acc = [[0.955, 0.98, 0.845, 0.6075, 0.8575, 0.975, 0.7975, 0.9475, 0.96, 0.9625, 0.9025, 0.9775],
# [0.9675, 0.98, 0.8425, 0.61, 0.91, 0.975, 0.8175, 0.96, 0.965, 0.945, 0.9625, 0.975],
# [0.975, 0.9825, 0.8825, 0.76, 0.985, 0.9625, 0.945, 0.965, 0.99, 0.9875, 0.99, 0.99],
# [0.9725, 0.9775, 0.8475, 0.56, 0.9075, 0.9725, 0.7875, 0.9575, 0.955, 0.9425, 0.965, 0.97],
# [0.98, 0.98, 0.8725, 0.76, 0.99, 0.9575, 0.9525, 0.96, 0.99, 0.9875, 0.985, 0.99],
# [0.88, 0.9075, 0.7375, 0.5, 0.2, 0.975, 0.8525, 0.875, 0.815, 0.825, 0.925, 0.9675],
# [0.7475, 0.91, 0.6625, 0.495, 0.5725, 0.975, 0.815, 0.875, 0.7275, 0.835, 0.915, 0.9375],
# [0.97, 0.925, 0.8225, 0.5675, 0.015, 0.975, 0.79, 0.94, 0.9475, 0.8925, 0.96, 0.97]]


In [13]:
# for engine in ['postgres', 'mssql']:

#     if engine == 'mssql':
#         baseline = mssql_accs
#     else:
#         baseline = postgres_accs

#     if engine == 'postgres':
#         acc0 = postgres_feature_set_0_acc
#         acc1 = postgres_feature_set_1_acc
#     else:
#         acc0 = mssql_feature_set_0_acc
#         acc1 = mssql_feature_set_1_acc
#     for i in range(len(present_idxs)):
#         for a0, a1, b in zip(acc0, acc1, baseline):
#             print(f"& {'%+.2f, %+.2f' % (a0[i] - b[i], a1[i] - b[i])} ".replace('0.', '.'), end=' ')
#         print("")
#     print('\n\n\n\n\n\n')

In [14]:
# for engine in ['postgres', 'mssql']:

#     if engine == 'mssql':
#         baseline = mssql_accs
#     else:
#         baseline = postgres_accs

#     if engine == 'postgres':
#         acc0 = postgres_feature_set_0_acc
#         acc1 = postgres_feature_set_1_acc
#     else:
#         acc0 = mssql_feature_set_0_acc
#         acc1 = mssql_feature_set_1_acc
#     for i in range(len(present_idxs)):
#         for a0, a1, b in zip(acc0, acc1, baseline):
# #             print(f"& {'%.2f, %.2f' % (a0[i], a1[i])} ".replace('0.', '.'), end=' ')
#             l = "& \\textbf{ %.2f } " % a0[i]
#             if a0[i] >= 0.90:  
#                 l = "& \\textbf{ %.2f } " % a0[i]
#             else:
#                 l = "&  %.2f " % a0[i]
#             print(l, end = '')

#         print("BREAK LINE")
#     print('\n\n\n\n\n\n')